In [1]:
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from datasets import BCDataDataset, PointsToGaussianHeatmap, collate_heatmap_points
from visualization import overlay_heatmap
from training import train
from models import NucleusLocalizationModel, heatmap_weighted_mse_loss

from utils.debug import print_info

import torch
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [2]:
with open("config.yaml", "r") as f:
    cfg = yaml.safe_load(f)


data_root = Path(cfg["paths"]["data_root"])
print(data_root)

/home/yeldos/Istanbul/obsidian_vault/work_AITU/IHC/DATASETS/BCData


In [3]:
print(Path(cfg["collab_paths"]["data_root"]))
print(Path(cfg["collab_paths"]["checkpoint_dir"]))

/content/drive/MyDrive/BCData
/content/BCData_Ki67/checkpoints


In [3]:
heatmap_generator = PointsToGaussianHeatmap(out_hw=(160,160), in_hw=(640,640), sigma=2.0)

In [4]:
dataset = BCDataDataset(root = data_root,
                        split="train",
                        target_transform = heatmap_generator)

In [ ]:
train_loader = DataLoader(
    dataset,
    batch_size=5,        # choose based on GPU memory (640×640 images are large)
    shuffle=True,
    num_workers=0,       # use 0 if debugging
    pin_memory=True,     # recommended when using GPU
    drop_last=True,       # optional, useful for BatchNorm
    collate_fn = collate_heatmap_points
)

In [6]:
model = NucleusLocalizationModel()
device = 'cuda'

In [7]:
losses = train(model=model, num_epochs=10, train_loader=train_loader, val_loader=None, loss_function=heatmap_weighted_mse_loss, forplot_img=None)

training start


  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 0


 10%|█         | 1/10 [02:46<24:54, 166.04s/it]

epoch: 1


 10%|█         | 1/10 [03:26<30:54, 206.03s/it]


KeyboardInterrupt: 

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
# model.train()
# for imgs, heatmaps, pos_points, neg_points in train_loader:
#     #print("imgs:");          print_info(imgs)             # (B, 3, 640, 640)
#     #print("heatmaps:"); print_info(heatmaps)      # (B, 2, 160, 160)
#     #print("pos_points:");   print_info(pos_points)        # list
#     #print("neg_points:");   print_info(neg_points)        # list



#     preds = model(imgs)

#     loss = heatmap_weighted_mse_loss(preds, heatmaps)


#     break

loss: 0.10676933825016022
